In [1]:
import os
import sys

path_root = os.path.dirname(os.getcwd())

if path_root not in sys.path:
    sys.path.append(path_root)

In [2]:
from src.generate.config import path_output_synthea, path_output_llm
from src.utils import write
import src.generate.synthea as synthea
import src.generate.llm as llm

In [3]:
output_synthea = synthea.run("./run_synthea", "-p", "1", "West Yorkshire")

In [4]:
# write(output_synthea, path_output_synthea)

In [5]:
model = "llama2"
template = """[INST]
<<SYS>>
You are a medical student answering an exam question about writing clinical notes for patients.
<</SYS>>

Keep in mind that your answer will be asssessed based on incorporating all the provided information and the quality of prose.

1. Use prose to write an example clinical note for this patient's doctor.
2. Use less than three sentences.
3. Do not provide a diagnosis or recommendations.
4. Use the following information:

{data}
[/INST]
"""

In [6]:
output_llm = llm.run(
    model,
    template,
    data_variable=output_synthea,
    # data_path=path_output_synthea,
)

Clinical Note:
Patient Matha Ratke presents today for follow-up evaluation of perennial allergic rhinitis with seasonal variation. The patient was born on April 11, 2005 and has been experiencing symptoms consistent with this condition, including nasal congestion, sneezing, and itchy eyes, throughout the spring and summer months. No diagnosis or recommendations are provided in this note.

In [7]:
# write(output_llm, path_output_llm)